# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..
import os, sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))

/Users/Tony/Other Docs/distilling-and-forgetting-in-large-pre-trained-models


In [3]:
from functools import partial
from tqdm.auto import tqdm

import torch
from transformers import pipeline
from transformers.models.whisper import (WhisperTokenizer,
                                         WhisperTokenizerFast,
                                         WhisperFeatureExtractor,
                                         WhisperForConditionalGeneration)
from datasets import load_dataset
import evaluate

from dataloader.collator import DataCollatorSpeechSeq2SeqWithPadding
from dataloader.preprocessing_train.preprocessing import prepare_dataset_fct
from evaluation.eval_dataset_name_to_dataset_group import EVAL_DATASET_NAME_TO_DATASET_GROUP

if torch.cuda.is_available():
    device = "cuda:0"
elif torch.backends.mps.is_available():  # for Apple Silicon
    device = torch.device("mps")
else:
    device = "cpu"

## Load dataset

# With `tiny`

## Load the model

In [4]:
pretrained_model_name_or_path = "openai/whisper-tiny"

model = WhisperForConditionalGeneration.from_pretrained(pretrained_model_name_or_path)
feature_extractor = WhisperFeatureExtractor.from_pretrained(pretrained_model_name_or_path)
tokenizer = WhisperTokenizerFast.from_pretrained(pretrained_model_name_or_path, language="english", task="transcribe")

model.generate = partial(model.generate, task="transcribe", max_length=255, use_cache=True)

In [5]:
dataset_name = "librispeech_dummy"
# dataset_name = "ami"

ds = EVAL_DATASET_NAME_TO_DATASET_GROUP[dataset_name]()[dataset_name]

if dataset_name == "ami":
    ds = ds.select(list(range(32)))

Found cached dataset librispeech_asr_dummy (/Users/Tony/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [6]:
prepare_dataset = partial(prepare_dataset_fct, tokenizer=tokenizer, feature_extractor=feature_extractor)
ds = ds.map(prepare_dataset, num_proc=4).with_format("pt")

Loading cached processed dataset at /Users/Tony/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b/cache-59ee67852172721a_*_of_00004.arrow


## Predict

In [7]:
x = ds[:5]

references = [elt.lower() for elt in x["text"]]

outputs = model.generate(x["input_features"], language="english")

transcribe_en = tokenizer.batch_decode(outputs, skip_special_tokens=True)

outputs = model.generate(x["input_features"], language="french")

transcribe_pseudo_french = tokenizer.batch_decode(outputs, skip_special_tokens=True)

outputs = model.generate(x["input_features"], task="translate", language="french")

translate_pseudo_fr_to_en = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [8]:
import pandas as pd
from IPython.display import display

# create the DataFrame
df = pd.DataFrame({'references': references,
                   'transcribe_en': transcribe_en,
                   'transcribe_pseudo_french': transcribe_pseudo_french,
                   'translate_pseudo_fr_to_en': translate_pseudo_fr_to_en
                  })

with pd.option_context('display.max_colwidth', None):
    display(df)

,references,transcribe_en,transcribe_pseudo_french,translate_pseudo_fr_to_en
0,mister quilter is the apostle of the middle classes and we are glad to welcome his gospel,Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.,Le plus de la chasse est de la classe et nous débrouillons l'air de l'air de la chasse.,Mr. Kfilter is the apostle of the middle classes and we are glad to welcome his gospel.
1,nor is mister quilter's manner less interesting than his matter,Nor is Mr. Quilters' manner less interesting than his matter.,N'orise Mr. Quilterre s'en m'en est intéressant thané le mâtre.,Nor is Mr. Quilters' manner less interesting than his matter.
2,he tells us that at this festive season of the year with christmas and roast beef looming before us similes drawn from eating and its results occur most readily to the mind,"He tells us that at this festive season of the year with Christmas and roast beef looming before us, similarly drawn from eating and its results occur most readily to the mind.",Il a dit que cette fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois-tu vu une fois,"He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly drawn from eating and its results occur most readily to the mind."
3,he has grave doubts whether sir frederick leighton's work is really greek after all and can discover in it but little of rocky ithaca,He has grave doubts whether Sir Frederick Layton's work is really Greek after all and can discover in it but little of Rocky Ithaca.,"Il y a un grave d'autres, où les frais de l'Athénent est vraiment très bien. Et on peut se retrouver dans un peu plus de l'Etat de Rochie Ithaca.","He has grave douts, whether Sir Frederick Layton's work is really Greek after all, and can discover in it but little of Rocky Ithaca."
4,linnell's pictures are a sort of up guards and at em paintings and mason's exquisite idylls are as national as a jingo poem mister birket foster's landscapes smile at one much in the same way that mister carker used to flash his teeth and mister john collier gives his sitter a cheerful slap on the back before he says like a shampooer in a turkish bath next man,"Lennils, pictures are a sort of upguards and atom paintings, and Mason's exquisite Idols are as national as a jingo poem. Mr. Birkut Foster's landscapes smile at one much in the same way that Mr. Karker used to flash his teeth. And Mr. John Colier gives his sitter a cheerful slap on the back before he says, like a shampoo or a turkish bath, next man,","Les nels, pictures, ont un peu de garages et des empêntings, et ce qui est un peu de garages, et ce qui est un peu de garages national comme une gingue. Le premier ministre de Bercut, il est un peu de la main, et il est un peu de la même chose que le ministre de Carquer, il est une fleur de taîte. Le ministre de John Collier, qui est un petit peu de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la fin de la f

# Compare all Whisper flavors

In [9]:
list_model_paths = [
    "openai/whisper-tiny",
    "openai/whisper-base",
    "openai/whisper-small",
    "openai/whisper-medium"
]

x = ds[0:1]
preds = {}

for pretrained_model_name_or_path in tqdm(list_model_paths):
    model = WhisperForConditionalGeneration.from_pretrained(pretrained_model_name_or_path)
    model.generate = partial(model.generate, task="transcribe", max_length=100, use_cache=True)
    outputs = model.generate(x["input_features"], language="french")
    transcribe_pseudo_french = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    preds[pretrained_model_name_or_path] = transcribe_pseudo_french

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
preds

{'openai/whisper-tiny': [" Le plus de la chasse est de la classe et nous débrouillons l'air de l'air de la chasse."],
 'openai/whisper-base': [' Mr. Quilter est le passé de la classe de la ville, et nous sommes glad de bienvenue son gosse-boule.'],
 'openai/whisper-small': [" Mr Quilter est l'époche de la classe moyenne et nous sommes heureux d'accueillir son gospel."],
 'openai/whisper-medium': [" Mr. Quilter est l'apostle des classes du milieu et nous sommes heureux de la bienvenue dans son Gospèle."]}